# Demonstrations

In [ ]:
using IsingModel
using BenchmarkTools
using Conda
using LinearAlgebra
using Plots
using PyCall
using Random, Distributions
using Revise
using SparseArrays

# Uncomment the following two lines the first time to run this demonstration.
#Conda.add("scipy")
#Conda.add("networkx")
const nx = pyimport("networkx")

In [ ]:
const scipy_sparse_find = pyimport("scipy.sparse")["find"]
function mysparse(Apy::PyObject)
    IA, JA, SA = scipy_sparse_find(Apy)
    return sparse(Int[i + 1 for i in IA], Int[i + 1 for i in JA], SA)
end

In [ ]:
const N = 64  # The number of nodes
const SIDE_LENGTH = (Int ∘ ceil ∘ sqrt)(N)

# Generate a square lattice with the periodic boundary condition by NetworkX.
G = nx.grid_2d_graph(SIDE_LENGTH, SIDE_LENGTH, periodic=true)
nx.set_edge_attributes(G, values=-1, name="weight")

adjacencyMatrix = mysparse(nx.adjacency_matrix(G))
bias = zeros(N)
const INITIAL_CONFIGURATION = 2 .* rand(Bernoulli(0.5), N) .- 1
spinSystem = SpinSystem(INITIAL_CONFIGURATION, adjacencyMatrix, bias)

In [ ]:
const MAX_STEPS = N^2
const INITIAL_TEMPERATURE = float(N)
const FINAL_TEMPERATURE = 0.0

#annealingSchedule(n) = (FINAL_TEMPERATURE - INITIAL_TEMPERATURE) / MAX_STEPS * n + INITIAL_TEMPERATURE
annealingSchedule(n) = INITIAL_TEMPERATURE^(-n)

function runAnnealer(Algorithm::Type{<:IsingModel.UpdatingAlgorithm}, spinSystem::SpinSystem, INITIAL_TEMPERATURE=-1.0)::Vector{Float64}
    if INITIAL_TEMPERATURE < 0.0
        return map(calcEnergy, takeSamples!(Algorithm(deepcopy(spinSystem)), MAX_STEPS, annealingSchedule))
    else
        return map(calcEnergy, takeSamples!(Algorithm(deepcopy(spinSystem), INITIAL_TEMPERATURE), MAX_STEPS, annealingSchedule))
    end
end

In [ ]:
@benchmark runAnnealer(GlauberDynamics, spinSystem, INITIAL_TEMPERATURE)

In [ ]:
plot(xlabel="MC steps", ylabel="Energy")
plot!(runAnnealer(AsynchronousHopfieldNetwork, spinSystem), label="Hopfield")
plot!(runAnnealer(GlauberDynamics, spinSystem, INITIAL_TEMPERATURE), label="Glauber")
plot!(runAnnealer(MetropolisMethod, spinSystem, INITIAL_TEMPERATURE), label="Metropolis")